In [9]:
import yfinance as yf
import pandas as pd

In [10]:
# Download S&P 500 data
sp500 = yf.Ticker("^GSPC")
sp500 = sp500.history(period="Max")
del sp500["Dividends"]
del sp500["Stock Splits"]
# del sp500["Volume"]

In [11]:
sp500['MA_10'] = sp500['Close'].rolling(window=10).mean()
sp500['MA_50'] = sp500['Close'].rolling(window=50).mean()

sp500['MA_Diff'] = sp500['MA_10'] - sp500['MA_50']

sp500.dropna(inplace=True)

sp500['Target'] = sp500['Close'].shift(-1)

sp500 = sp500[:-1]
sp500

,Open,High,Low,Close,Volume,MA_10,MA_50,MA_Diff,Target
Date,,,,,,,,,
1928-03-13 00:00:00-05:00,17.920000,17.920000,17.920000,17.920000,0,17.617000,17.473000,0.144000,17.930000
1928-03-14 00:00:00-05:00,17.930000,17.930000,17.930000,17.930000,0,17.684000,17.478400,0.205600,18.070000
1928-03-15 00:00:00-05:00,18.070000,18.070000,18.070000,18.070000,0,17.761000,17.484600,0.276400,18.260000
1928-03-16 00:00:00-05:00,18.260000,18.260000,18.260000,18.260000,0,17.857000,17.495400,0.361600,18.360001
1928-03-19 00:00:00-05:00,18.360001,18.360001,18.360001,18.360001,0,17.936000,17.511600,0.424400,18.459999
...,...,...,...,...,...,...,...,...,...
2024-03-21 00:00:00-04:00,5253.430176,5261.100098,5240.660156,5241.529785,4207730000,5164.385938,5000.652178,163.733760,5234.180176
2024-03-22 00:00:00-04:00,5242.479980,5246.089844,5229.870117,5234.180176,3374700000,5175.434961,5009.666777,165.768184,5218.189941
2024-03-25 00:00:00-04:00,5219.520020,5229.089844,5216.089844,5218.189941,3331360000,5185.459961,5018.425771,167.034189,5203.580078


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Select features and target
X = sp500[['MA_10', 'MA_50', 'MA_Diff']]
y = sp500['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
predictions = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 351.016337040356
